In [1]:
import jax

# https://jax.readthedocs.io/en/latest/notebooks/Distributed_arrays_and_automatic_parallelization.html#generating-random-numbers
jax.config.update('jax_threefry_partitionable', True)

from jax import random, numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax

from jax.sharding import Mesh, PartitionSpec
from jax.experimental import mesh_utils

from typing import Callable
import numpy as np
from functools import partial
from helpers import ShardedTrainer, MeshManager, get_batch_gen

from hyena import HyenaOperator

2023-10-23 23:00:42.294269: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
key = random.PRNGKey(0)

I0000 00:00:1698094846.926878 1350637 pjrt_api.cc:98] GetPjrtApi was found for tpu at /home/amir/.venv311/lib/python3.11/site-packages/libtpu/libtpu.so
I0000 00:00:1698094846.926959 1350637 pjrt_api.cc:67] PJRT_Api is set for device type tpu
I0000 00:00:1698094846.926964 1350637 pjrt_api.cc:72] PJRT plugin for tpu has PJRT API version 0.30. The framework PJRT API version is 0.30.
I0000 00:00:1698094849.939847 1350637 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


In [8]:
# causality check

layer = HyenaOperator(
    max_len=256,
    d_model=128,
    pos_embed_dim=7,
    filter_features=64,
    num_filter_layers=4
)

key, init_key, drop_key, data_key = random.split(key, 4)

x = random.normal(data_key, (1, 256, 128))

params = layer.init({"params": init_key, "dropout": drop_key}, x)

@jax.grad
def out_grads(x):
    logits = layer.apply(params, x, rngs={"dropout": drop_key}, train=True)
    # Get some scalar-valued output with respect to which to evaluate grad.
    out = jnp.sum(logits[:, 10, :])
    return out


grads = out_grads(x)
# Expectation: grad wrt sequence items with pos > 10 should be negligible, with pos <= 10 significant.
print(jnp.sum(jnp.abs(grads[0,11:,:])))
print(jnp.sum(jnp.abs(grads[0,:11,:])))

0.0003403878
475.82037


In [9]:
# performance test

jit_apply = jax.jit(lambda x: layer.apply(params, x, rngs={"dropout": drop_key}, train=True))

# warmup
jit_apply(x)

%timeit jit_apply(x).block_until_ready()

338 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
# standard transformer decoder

class HyenaDecoderLayer(nn.Module):
    features: int
    max_len: int
    filter_features: int
    num_filter_layers: int
    pos_embed_dim: int
    order: int
    hidden_features: int
    dropout: float = 0.0

    @nn.compact
    def __call__(self, x, train: bool = True):
        residual = x
        x = nn.LayerNorm()(x)
        x = HyenaOperator(
            self.max_len,
            self.features,
            self.pos_embed_dim,
            self.filter_features,
            self.num_filter_layers,
            self.order,
            dropout=self.dropout,
        )(x, train=train)
        x = nn.Dropout(rate=self.dropout)(x, deterministic=not train)
        x = residual + x

        residual = x
        x = nn.LayerNorm()(x)
        x = nn.Dense(self.hidden_features)(x)
        x = nn.gelu(x)
        x = nn.Dense(self.features)(x)
        x = nn.Dropout(rate=self.dropout)(x, deterministic=not train)
        x = residual + x

        return x


class HyenaDecoder(nn.Module):
    embedding: nn.Module
    layer_func: Callable
    n_layers: int
    dropout: float = 0.0

    @nn.compact
    def __call__(self, tokens, train: bool = True):
        embeds = self.embedding(tokens)
        embeds = nn.Dropout(rate=self.dropout)(embeds, deterministic=not train)

        for _ in range(self.n_layers):
            embeds = self.layer_func(dropout=self.dropout)(embeds, train=train)
        
        embeds = nn.LayerNorm()(embeds)
        logits = self.embedding.attend(embeds)
        return logits * 1 / jnp.sqrt(self.embedding.features)

In [11]:
n_dim = 128
n_layers = 6
seq_len = 512
embed_dim = 7
filter_features = 64
num_filters_layers = 4
order = 2
dropout = 0.1

embed_fn = partial(nn.Embed, num_embeddings=65, features=n_dim)
layer = partial(
    HyenaDecoderLayer,
    features=n_dim,
    max_len=seq_len,
    filter_features=filter_features,
    num_filter_layers=num_filters_layers,
    pos_embed_dim=embed_dim,
    order=order,
    hidden_features=n_dim * 4,
)
model = HyenaDecoder(
    embedding=embed_fn(),
    layer_func=layer,
    n_layers=n_layers,
    dropout=dropout
)

In [12]:
class ShakespeareTrainer(ShardedTrainer):
    def get_loss_function(self):
        def calculate_loss(params, rng, batch, train):
            inp_data, labels = batch

            rng, dropout_apply_rng = random.split(rng)
            logits = self.model.apply({'params': params}, inp_data, train=train, rngs={'dropout': dropout_apply_rng})
            loss = optax.softmax_cross_entropy_with_integer_labels(logits, labels).mean()
            acc = (logits.argmax(axis=-1) == labels).mean()
            return loss, (acc, rng)
        return calculate_loss
    
    def init_model(self, exmp_batch):
        self.rng = jax.random.PRNGKey(self.seed)
        self.rng, init_rng, dropout_init_rng = jax.random.split(self.rng, 3)

        # learning rate schedule and optimizer
        sched = optax.warmup_cosine_decay_schedule(
            init_value=0,
            peak_value=1e-3,
            warmup_steps=100,
            decay_steps=self.max_iters,
            end_value=7e-4,
        )
        optimizer = optax.chain(optax.clip_by_global_norm(1.0),
                                optax.adamw(sched, weight_decay=0.1, b2=0.98))

        # initialization function
        def init_fn(init_rng, dropout_init_rng, x, model, optimizer):
            inp_data = x[0]
            variables = self.model.init({'params': init_rng, 'dropout': dropout_init_rng}, inp_data, train=True)
            params = variables['params']
            state = train_state.TrainState.create(
                apply_fn=model.apply,
                params=params,
                tx=optimizer
            )
            return state

        # get init_fn output sharding (state)
        abstract_vars, logical_state_spec = MeshManager.get_var_sharding(init_fn,
                                                                         init_rng,
                                                                         dropout_init_rng,
                                                                         exmp_batch,
                                                                         model=self.model,
                                                                         optimizer=optimizer)
        # convert logical to physical sharding according to rules
        rules = (('batch', 'data'),)
        state_sharding = self.mesh_manager.logical_to_mesh(logical_state_spec, rules)

        jitted_init = jax.jit(
            init_fn, static_argnums=(3,4),
            in_shardings=(None, None, self.data_sharding),
            out_shardings=state_sharding
        )
        
        initialized_state = jitted_init(init_rng, dropout_init_rng, exmp_batch, model, optimizer)

        return initialized_state, state_sharding

In [13]:
device_count = jax.device_count()

device_mesh = mesh_utils.create_device_mesh((8,1))
mesh = Mesh(devices=device_mesh, axis_names=('data', 'model'))
mesh_manager = MeshManager(mesh)
print(mesh)

data_shard_spec = PartitionSpec('data', None)
data_sharding = mesh_manager.mesh_sharding(data_shard_spec)

Mesh(device_ids=array([[0],
       [1],
       [2],
       [3],
       [6],
       [7],
       [4],
       [5]]), axis_names=('data', 'model'))


In [14]:
from urllib import request

# Test on the tiny Shakespeare dataset.
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = request.urlopen(url)
text = response.read().decode("utf-8")

vocab = sorted(set(text))
char2token = {c: i for (i, c) in enumerate(vocab)}
token2char = {i: c for (i, c) in enumerate(vocab)}
tokens = jnp.array([char2token[c] for c in text])

split_idx = int(0.9*len(tokens))
train, test = tokens[:split_idx], tokens[split_idx:]

In [15]:
def get_batch(idxs, data, seq_len):
    tok_idxs = jnp.array(idxs)[:, jnp.newaxis] + jnp.arange(seq_len)
    input_tokens = data[tok_idxs]
    target_tokens = data[tok_idxs + 1]

    batch = (input_tokens, target_tokens)
    sharded_batch = mesh_manager.shard_data(batch, data_sharding)

    return sharded_batch

get_train_batch = partial(get_batch, data=train, seq_len=seq_len)
get_test_batch = partial(get_batch, data=test, seq_len=seq_len)

exmp_batch = get_batch(np.arange(0, device_count), train, seq_len)
jax.debug.visualize_array_sharding(exmp_batch[0])

                                     TPU 0                                      
                                                                                
                                     TPU 1                                      
                                                                                
                                     TPU 2                                      
                                                                                
                                     TPU 3                                      
                                                                                
                                     TPU 6                                      
                                                                                
                                     TPU 7                                      
                                                                                
                                     TPU 4                                      
                                                                                
                                     TPU 5                                      
                                                                                

In [16]:
n_epochs = 40
batch_size = 64

# Don't choose inputs from the last seq_len indices as targets would exceed train/test size.
train_size = train.shape[0]-(seq_len+1)
test_size = test.shape[0]-(seq_len+1)

# Don't iterate over every single item in an epoch (unnecessarily large overlap between batches).
train_steps_per_epoch = train_size//(batch_size*256)
test_steps_per_epoch = test_size//(batch_size*25)

get_train_gen = partial(get_batch_gen,
                        train_size,
                        get_train_batch,
                        batch_size,
                        shuffle=True,
                        steps_per_epoch=train_steps_per_epoch)
get_val_gen = partial(get_batch_gen,
                        test_size,
                        get_test_batch,
                        batch_size,
                        shuffle=True,
                        steps_per_epoch=test_steps_per_epoch)

In [17]:
key, train_key = random.split(key)

num_train_iters = train_steps_per_epoch * n_epochs

# Create a trainer module with specified hyperparameters
trainer = ShakespeareTrainer(model,
                    model_name='ShakespeareHyena',
                    exmp_batch=exmp_batch,
                    max_iters=num_train_iters,
                    data_sharding=data_sharding,
                    mesh_manager=mesh_manager)

/home/amir/work/tests/hyena_own/helpers.py:81: UserWarning: WARNING: Checkpoint directory already exists for this model. Model will not be saved.
  warnings.warn('WARNING: Checkpoint directory already exists for this model. Model will not be saved.')


In [18]:
with mesh:
    trainer.train_model(get_train_gen, get_val_gen, num_epochs=n_epochs)

val_gen, train_key = get_val_gen(key=train_key)
val_acc, val_loss = trainer.eval_model(val_gen)

# Bind parameters to model for easier inference
trainer.model_bd = trainer.model.bind({'params': trainer.state.params})

  0%|          | 0/40 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Epoch 1, train loss: 4.0149312019348145, accuracy: 0.11218111962080002
Epoch 1, val loss: 3.791551113128662, accuracy: 0.17125403881072998


Training: 0it [00:00, ?it/s]

Epoch 2, train loss: 3.501777410507202, accuracy: 0.25221025943756104
Epoch 2, val loss: 3.1810083389282227, accuracy: 0.3415014147758484


...

Epoch 38, train loss: 1.2349002361297607, accuracy: 0.6131226420402527
Epoch 38, val loss: 1.4228782653808594, accuracy: 0.5760365128517151


Training: 0it [00:00, ?it/s]

Epoch 39, train loss: 1.2320034503936768, accuracy: 0.6140176653862
Epoch 39, val loss: 1.43060302734375, accuracy: 0.5749737024307251


Training: 0it [00:00, ?it/s]

Epoch 40, train loss: 1.2268190383911133, accuracy: 0.6152999401092529
Epoch 40, val loss: 1.4329246282577515, accuracy: 0.5764301419258118


In [20]:
@jax.jit
def pred(inp, key):
    logits = trainer.model_bd(inp, train=False)
    # Sample instead of simply taking argmax to reduce repetition.
    token_idx = random.categorical(key, logits, axis=-1)
    return token_idx

# warmup
pred(jnp.zeros((1, seq_len), dtype=jnp.int32), key).shape

(1, 512)

In [22]:
cond_text = "ROMEO:"
cond_tokens = [char2token[c] for c in cond_text]
cur_in = jnp.array([cond_tokens])
cur_idx = cur_in.shape[-1]
cur_in = np.pad(cur_in, ((0,0),(0,seq_len-cur_idx)))

sample_key = random.PRNGKey(3)

# number of chars to generate
num_gen = 800

for i in range(0, num_gen):
    sample_key = random.fold_in(sample_key, i)

    pred_tokens = pred(cur_in, sample_key)
    cur_pred_token = pred_tokens[0,cur_idx-1]

    if cur_idx < seq_len:
        cur_in[0,cur_idx] = cur_pred_token
        cur_idx += 1
    else:
        cur_in = np.roll(cur_in, shift=-1, axis=-1)
        cur_in[0,cur_idx-1] = cur_pred_token
    
    cond_tokens.append(int(cur_pred_token))

print(''.join([token2char[c] for c in cond_tokens]))

ROMEO:
Alas, my father, and I know come; let's prejoice
To die for reply, and they there, like man
Where will abuse off: mine arst Abury,'
I dead to. This true that was all eyes,
Passing him till 'we must speak of monsta's maid,
Should not enbear him! hear the duke, whood they are
commend to thee to do it?
And send not Barnardine, but giving a scorn
To find a perfect.

MENENIUS:
Tell him, you have sead'd to stand up to death.
You shall not indeed, say 'stain.
Long that was won; I'll not such a man!
shut it not that beheld how all I like thee,
Ratcliff, tribunes no dear silence we hereafter.

PRINCE EDWARD:
Most ragely from him succession:
I pretty troopque, Richmond the gates to say,
By curity of the matters but herein;
Which with a few day in the hire infect.
Look your silence than my name in k
